In [2]:
import cv2
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import random

def select_white_yellow(image):
    hls = convert_hls(image)
    # white color mask
    lower = np.uint8([  100, 160,7])
    upper = np.uint8([160, 200, 60])
    white_mask = cv2.inRange(hls, lower, upper)
    
    hsv = convert_hsv(image)
    # yellow color mask HSV
    lower = np.uint8([ 40, 20, 0])
    upper = np.uint8([ 110, 255, 255])
    #lower = np.uint8([ 30, 20, 0])
    #upper = np.uint8([ 80, 100, 100])
    
    yellow_mask = cv2.inRange(hsv, lower, upper)
    # combine the mask
    mask = cv2.bitwise_or(white_mask, yellow_mask)

    return cv2.bitwise_and(image, image, mask = mask)
def convert_hls(image):
    return cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
def convert_hsv(image):
    return cv2.cvtColor(image, cv2.COLOR_RGB2HSV)

def select_region(image):
    # first, define the polygon by vertices
    rows, cols = image.shape[:2]
    bottom_left  = [cols*0.35, rows*0.95]
    top_left     = [cols*0.6, rows*0.3]
    bottom_right = [cols*0.6, rows*0.95]
    top_right    = [cols*0.35, rows*0.3] 
    # the vertices are an array of polygons (i.e array of arrays) and the data type must be integer
    vertices = np.array([[bottom_left, bottom_right, top_left, top_right]], dtype=np.int32)
    mask = np.zeros_like(image)
    if len(img.shape)==3:
        cv2.fillPoly(mask, vertices, 255)
    else:
        cv2.fillPoly(mask, vertices, (255,)*mask.shape[2])   

    return cv2.bitwise_and(image, mask)

In [3]:
def getcell(ind,nd):
    x,y=ind
    cell=np.zeros(nd)
    theta=np.array(range(nd))/nd*np.pi
    theta1=np.arctan(y/x)
    l=(x**2+y**2)**0.5
    for i in range(nd):
        cell[i]=abs(np.cos(theta[i]-theta1)*l)
    return cell
def redefine(res,r=0.7):
    ma=res.max()
    line=np.where(res>r*ma)
    return line

In [4]:
def gather(lines):
    line=np.array(lines)
    nl=len(line[0])
    #print(nl)
    center=-np.ones([nl,2])
    num=np.zeros(nl)
    c=0
    for i in range(nl):
        #print(center)
        if c==0:
            center[c,:]=line[:,i]
            num[c]=num[c]+1
            c=c+1
            #print('new:{}'.format(line[:,i]))
        else:
            res=((center-line[:,i])**2).sum(axis=1)
            resmin=np.min(res)
            if resmin>300:
                center[c,:]=line[:,i]
                num[c]=num[c]+1
                c=c+1
                #print('new:{}'.format(line[:,i]))
            else :
                pos=np.where(res==resmin)[0][0]
                if center[pos,0]>line[0,i]:
                    center[pos,:]=line[0,i]
                num[pos]=num[pos]+1
                #print('new added:{}'.format(line[:,i]))
    ind=(-num).argsort()[0:2]
    fline=center[ind,:]
    return fline
def linedraw(img,line):
    nl=len(line)
    ind1=[]
    ind2=[]
    for i in range(nl):
        r,theta=line[i]
        theta=theta/360*np.pi
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a*r
        y0 = b*r
        x1 = int(x0 + 100*(-b))
        y1 = int(y0 + 100*(a))
        x2 = int(x0 - 100*(-b))
        y2 = int(y0 - 100*(a))
        #cv2.line(img,(y1,x1),(y2,x2),50,2)
        ind1.append([y1,x1])
        ind2.append([y2,x2])
    indm=[list(np.array(ind1).mean(axis=0).astype(int)),list(np.array(ind2).mean(axis=0).astype(int))]
    #cv2.line(img,tuple(indm[0]),tuple(indm[1]),100,2)
    #cv2.imshow('bird',img)
    return ind1,ind2,indm

In [5]:
def hough(edges,prel=[]):
    ac=0.5
    edgep=np.array(np.where(edges>0)).T
    l=len(edgep)
    nd=round(180/ac)
    cells=np.zeros([l,nd])
    for i in range(l):
        cells[i,:]=getcell(edgep[i],nd)
    cells=cells.astype('int')
    m=cells.max()
    res=np.zeros([m,nd])
    for i in range(l):
        for j in range(nd):
            r=cells[i,j]
            d=j
            res[r-1,j]=res[r-1,j]+1
    res=(res/res.max())
    lines=redefine(res,0.5)
    line=gather(lines)
    if (line[1]**2).sum()<10:
        ind1,ind2,indm=prel
        #ind1,ind2,indm=linedraw(gray2,line)
    else:
        ind1,ind2,indm=linedraw(gray2,line)
    if abs(line[0,1]-line[1,1])>15:
        ind1,ind2,indm=prel
    return ind1,ind2,indm
def renew(prel,newl,s=0.3):
    p=np.array(prel)
    n=np.array(newl)
    res=(n*s+p*(1-s)).astype(int)
    resl=([list(np.array(res)[0,0]),list(np.array(res)[0,1])],
          [list(np.array(res)[1,0]),list(np.array(res)[1,1])],
          [list(np.array(res)[2,0]),list(np.array(res)[2,1])])
    return resl
def inv(end1,Mi):
    resm=Mi.dot(end1)
    resf=resm[0:2]/resm[2]
    return tuple(resf.astype(int))
def inverse(prel,Mi):
    end1=list(prel[1][0]+np.array([0,0]))+[1]
    end2=list(prel[1][1]-np.array([0,0]))+[1]
    end3=list(prel[2][1])+[1]
    start1=list(prel[0][0]+np.array([0,0]))+[1]
    start2=list(prel[0][1]-np.array([0,0]))+[1]
    start3=list(prel[2][0])+[1]
    
    res=(inv(start1,Mi),
        inv(start2,Mi),
        inv(start3,Mi),
        inv(end1,Mi),
        inv(end2,Mi),
        inv(end3,Mi),)
    return res
def getline(vl,ox,oy):
    va=np.array(vl)
    va=va+[ox,oy]
    #extend
    va[3]=va[3]+va[3]-va[0]
    va[4]=va[4]+va[4]-va[1]
    va[5]=va[5]+va[5]-va[2]
    res=(tuple(va[0]),tuple(va[1]),tuple(va[2]),tuple(va[3]),tuple(va[4]),tuple(va[5]))
    return res

In [7]:
# Create a VideoCapture object and read from input file
# If the input is the camera, pass 0 instead of the video file name
#vedioname = input("vedio name: ")
vedioname = 't1.mp4'
cap = cv2.VideoCapture(vedioname)
flag=0
pro=0.3
# Check if camera opened successfully
if (cap.isOpened()== False): 
    print("Error opening video stream or file")

# Read until video is completed
while(cap.isOpened()):
# Capture frame-by-frame
    ret, frame = cap.read()
    

    # main operation goes here
    if ret == True:
        frame = cv2.resize(frame, (640, 480))
        frame0=frame[:,:]
        gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        
        r,c,n = frame.shape
        midx = int(c/2)
        midy = int(r/2)
        offsetx = 80
        offsety = 100
        ox=midx-offsetx+30
        oy=midy-offsety
        ex=midx+offsetx-10
        ey=midy+offsety
        frame = frame[ox:ex,oy:ey]

        #equlizer in rgb
        img_yuv = cv2.cvtColor(frame, cv2.COLOR_BGR2YUV)
        # equalize the histogram of the Y channel
        img_yuv[:,:,0] = cv2.equalizeHist(img_yuv[:,:,0])
        # convert the YUV image back to RGB format
        img = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)
        
        #converting grayscale and resize
        img = select_white_yellow(img)
        
        #img = cv2.cvtColor(frame, cv2.COLOR_RGB2HSV)
        

        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        cropx,cropy = gray.shape
        #print(cropx,cropy)
        
        
        #birds eye transformation
        ofho = 85
        src = np.float32([[0, cropx], [cropy, cropx], [0, 0], [cropy, 0]])
        dst = np.float32([[ofho, cropx], [cropy-ofho, cropx], [0, 0], [cropy, 0]])
        M = cv2.getPerspectiveTransform(src, dst) # The transformation matrix
        Mi = cv2.getPerspectiveTransform(dst, src) # The transformation matrix
        #cv2.imshow('Camera',gray)
        gray = cv2.warpPerspective(gray, M, (cropy, cropx))
        #cv2.imshow('Bird eye',gray)
        #cv2.waitKey(0)
        #cutof = 60
        #gray = gray[0:cropx,cutof:cropy-cutof]
        #gray = cv2.warpPerspective(gray, Mi, (cropy, cropx))
                
        #filter region
        gray = select_region(gray)
        kernel = np.ones((2,2),np.uint8)
        gray = cv2.dilate(gray,kernel,iterations = 1)
        #kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(2, 2))
        #gray = cv2.erode(gray,kernel,iterations = 1)
        #canny edge detection
        gray = cv2.Canny(gray,80,150,apertureSize = 3)
        
        #dilation process
        #kernel = np.ones((2,2),np.uint8)
        #gray = cv2.dilate(gray,kernel,iterations = 1)
        #threshold
        retse, gray = cv2.threshold(gray,50,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
        #lsd line detection
        segd = cv2.createLineSegmentDetector(0)
        dlines = segd.detect(gray)[0]
        gray = segd.drawSegments(gray*0,dlines)
        gray2 = cv2.cvtColor(gray,cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(gray2,50,150,apertureSize = 3)
        if flag==0:
            newl=hough(edges,[])
            prel=newl
            flag=1
        else:
            newl=hough(edges,prel)
        prel=renew(prel,newl,1)
        #prel=newl
        cv2.line(edges,tuple(prel[0][0]),tuple(prel[1][0]),100,2)
        cv2.line(edges,tuple(prel[0][1]),tuple(prel[1][1]),100,2)
        cv2.line(edges,tuple(prel[2][0]),tuple(prel[2][1]),150,2)
        #cv2.imshow('bird',edges)
        bird=cv2.cvtColor(edges,cv2.COLOR_GRAY2RGB)
        sb=bird.shape
        bxs=480-sb[0]
        bys=640-sb[1]
        frame0[0:sb[0],bys:640]=bird
        vl=inverse(prel,Mi)
        #lsd(gray)
        
        gray1=cv2.warpPerspective(gray, Mi, (cropy, cropx))
        cv2.line(gray1,vl[0],vl[3],[0,0,255],2)
        cv2.line(gray1,vl[1],vl[4],[0,0,255],2)
        cv2.line(gray1,vl[2],vl[5],[0,255,0],2)
        #cv2.imshow('real',gray1)
        fl=getline(vl,oy-5,ox+8)
        cv2.line(frame0,fl[0],fl[3],[0,0,255],4)
        cv2.line(frame0,fl[1],fl[4],[0,0,255],4)
        cv2.line(frame0,fl[2],fl[5],[0,255,255],2)
        #frame0[ox:ex,oy:ey,:]=frame0[ox:ex,oy:ey,:]+gray1
        font=cv2.FONT_HERSHEY_SIMPLEX
        of=(np.array(prel[2][0])-np.array([prel[2][1]]))[0]
        theta=np.arctan(of[0]/of[1])*180/np.pi
        frame0=cv2.putText(frame0,str(theta),(450,140),font,0.5,(0,0,0),2)
        cv2.imshow('Frame',frame0)
           
        if cv2.waitKey(5) & 0xFF == ord('q'):
            break
    else:
        break
    # Press Q on keyboard to exitq
    


# When everything done, release the video capture object
cap.release()
 
# Closes all the frames
cv2.destroyAllWindows()